In [36]:
import heapq
import numpy as np
import xml.etree.ElementTree as ET

from pprint import pprint

In [37]:
def read_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    params = dict()
    
    def get_grid():
        rows = []
        for i in root.findall("map/grid/row"):
            rows.append([_ for _ in map(int, i.text.split(' '))])
        return rows
    
    # get parametrs from xml
    params["width"] = int(root.find("map/width").text)
    params["height"] = int(root.find("map/height").text)
    params["startx"] = int(root.find("map/startx").text)
    params["starty"] = int(root.find("map/starty").text)
    params["finishx"] = int(root.find("map/finishx").text)
    params["finishy"] = int(root.find("map/finishy").text)
    params["grid"] = get_grid()
    params["searchtype"] = root.find('algorithm/searchtype').text # "astar", "theta", "jp_search", "bfs", "dijkstra"
    params["metrictype"] = root.find('algorithm/metrictype').text # "euclidean", "diagonal", "manhattan", "chebyshev"
    params["breakingties"] = root.find('algorithm/breakingties').text # "g-min", "g-max"
    params["hweight"] = float(root.find('algorithm/hweight').text)
    params["allowdiagonal"] = True if root.find('algorithm/allowdiagonal').text=='true' else False
    params["cutcorners"] = True if root.find('algorithm/cutcorners').text=='true' else False
    params["allowsqueeze"] = True if root.find('algorithm/allowsqueeze').text=='true' else False
    params["loglevel"] = float(root.find('options/loglevel').text)
    params["logpath"] = root.find('options/logpath').text
    params["logfilename"] = root.find('options/logfilename').text
    
    return params

In [38]:
xml_path = "./data/example.xml"
params = read_xml(xml_path)
pprint(params["searchtype"])
pprint(params["grid"])

'astar'
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 

In [39]:
class Grid():
    def __init__(self, weigth, height, grid):
        self.w = weigth
        self.h = height
        self.grid = grid
        self.walls = [(i,j) for i in range(len(self.grid))
                            for j in range(len(self.grid[i])) if self.grid[i][j]]
    
    
    def is_inside(self, point):
        return 0 <= point[0] < self.w and 0 <= point[1] < self.h
    
    
    def get_value(self, point):
        return 0 if point not in self.walls else 1

    
    def is_empty_diagonal(self, from_point, to_point):
        neighbor1 = self.get_value((to_point[0], from_point[1]))
        neighbor2 = self.get_value((from_point[0], to_point[1]))
        return not neighbor1 and not neighbor2
    
    
    def neighbors(self, point, allow_diagonal=False, cut_corners=False, allow_sq=False):
        x = point[0]
        y = point[1]
        
        neighbors = [(x + 1, y),
                     (x - 1, y),
                     (x, y + 1),
                     (x, y - 1)]
        
        diagonal_neighbors = [(x - 1, y - 1),
                              (x - 1, y + 1),
                              (x + 1, y - 1),
                              (x + 1, y + 1)] 
        if not cut_corners:
            for dif_x in [1, -1]:
                for dif_y in [1, -1]:
                    if self.get_value((x + dif_x, y)) and self.get_value((x, y + dif_y)):
                        diagonal_neighbors.remove((x + dif_x, y + dif_y))

        if not allow_sq:
            for dif_x in [1, -1]:
                for dif_y in [1, -1]:
                    if self.get_value((x + dif_x, y)) or self.get_value((x, y + dif_y)):
                        diagonal_neighbors.remove((x + dif_x, y + dif_y))
            
        if allow_diagonal:
            neighbors += diagonal_neighbors
            
        neighbors = list(filter(self.is_inside, neighbors)) 
        neighbors = list(filter(not self.get_value, neighbors))
        return neighbors
    
    
    def cost(self, start_point, end_point, method='euclidean'):
        return pow(pow((start_point[0] - end_point[0]), 2) + pow((start_point[1] - end_point[1]), 2), 0.5)

In [40]:
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]